In [1]:
import numpy as np
import pandas as pd
from arch import arch_model

In [2]:
df_air_liquide = pd.read_csv("Data/air_liquide.csv", parse_dates=["Date"])

In [5]:
import matplotlib.pyplot as plt
from arch import arch_model
# Calcul des rendements log
df_air_liquide["log_return"] = (df_air_liquide["Open"] / df_air_liquide["Open"].shift(1)).apply(np.log)
returns = df_air_liquide["log_return"].dropna()

In [7]:
# Rendements log et rescaling x100
df_air_liquide["log_return"] = 100 * (df_air_liquide["Open"] / df_air_liquide["Open"].shift(1)).apply(np.log)
returns = df_air_liquide["log_return"].dropna()

# Calibration du GARCH(1,1)
model = arch_model(returns, vol="GARCH", p=1, q=1, dist="normal")
result = model.fit(disp="off")
print(result.summary())

                     Constant Mean - GARCH Model Results                      
Dep. Variable:             log_return   R-squared:                       0.000
Mean Model:             Constant Mean   Adj. R-squared:                  0.000
Vol Model:                      GARCH   Log-Likelihood:               -24401.0
Distribution:                  Normal   AIC:                           48810.0
Method:            Maximum Likelihood   BIC:                           48841.0
                                        No. Observations:                17481
Date:                Wed, Jul 23 2025   Df Residuals:                    17480
Time:                        20:22:38   Df Model:                            1
                                 Mean Model                                 
                 coef    std err          t      P>|t|      95.0% Conf. Int.
----------------------------------------------------------------------------
mu             0.0246  7.006e-03      3.509  4.490e-04 [1.

# 📈 Calibration d’un Modèle GARCH(1,1)

Ce notebook réalise une calibration économétrique standard d’un modèle **GARCH(1,1)** appliqué à une série de rendements financiers.

Le modèle est estimé via une approche de **Maximum de Vraisemblance (MLE)**.

---

## 1. Modèle estimé

Le code utilisé :

```python
model = arch_model(returns, vol="GARCH", p=1, q=1, dist="normal")
result = model.fit(disp="off")
```

correspond au modèle suivant :

### **GARCH(1,1) avec innovations gaussiennes**

---

## 2. Formulation mathématique

### 2.1 Équation des rendements

$$
r_t = \mu + \varepsilon_t
$$

$$
\varepsilon_t = \sigma_t z_t, \quad z_t \sim \mathcal{N}(0,1)
$$

où :

- \( r_t \) : rendement (log-return)  
- \( \mu \) : moyenne des rendements  
- \( \varepsilon_t \) : innovation aléatoire  
- \( \sigma_t^2 \) : variance conditionnelle  

---

### 2.2 Équation de la variance conditionnelle

$$
\sigma_t^2 = \omega + \alpha \varepsilon_{t-1}^2 + \beta \sigma_{t-1}^2
$$

C’est cette équation qui modélise la **dynamique de la volatilité**.

---

## 3. Signification des paramètres

| Paramètre | Rôle | Interprétation financière |
|----------|------|---------------------------|
| **μ (mu)** | Moyenne des rendements | Drift moyen du marché |
| **ω (omega)** | Niveau structurel | Variance de long terme |
| **α (alpha)** | Effet ARCH | Réaction aux chocs récents |
| **β (beta)** | Effet GARCH | Mémoire de la volatilité passée |

---

## 4. Persistance de la volatilité

$$
\alpha + \beta = \text{persistance}
$$

| Valeur | Signification |
|--------|--------------|
| < 0.8 | Faible mémoire |
| 0.8–0.95 | Marché normal |
| > 0.95 | Volatilité très persistante |
| = 1 | IGARCH (pas de retour à la moyenne) |

---

## 5. Variance de long terme

$$
\sigma^2_{\infty} = \frac{\omega}{1 - \alpha - \beta}
$$

Elle représente la volatilité vers laquelle le processus converge.

---

## 6. Méthode d’estimation

Le modèle est calibré par :

### **Maximum Likelihood Estimation (MLE)**

Sous l’hypothèse :

$$
z_t \sim \mathcal{N}(0,1)
$$

La log-vraisemblance maximisée est :

$$
\mathcal{L} = -\frac{1}{2} \sum_t \left( \log \sigma_t^2 + \frac{\varepsilon_t^2}{\sigma_t^2} \right)
$$

L’optimisation est :

- Non linéaire  
- Numérique  
- Résolue via des algorithmes quasi-Newton (BFGS / L-BFGS)

---

## 7. Lecture de la sortie `summary()`

| Élément | Interprétation |
|--------|----------------|
| coef | Valeur estimée |
| std err | Incertitude |
| t | Statistique de test |
| P>|t| | Significativité |

Si **α** et **β** sont significatifs → volatilité conditionnelle avérée.

---

## 8. Effet de la mise à l’échelle des rendements

$$
r_t = 100 \log\left(\frac{P_t}{P_{t-1}}\right)
$$

alors :

- Les paramètres deviennent plus stables numériquement  
- \( \omega \) change d’échelle (variance × 10 000)

---

## 9. Interprétation économique intuitive

| Situation | Signification |
|-----------|--------------|
| α élevé | Marché très sensible aux news |
| β élevé | Volatilité persistante |
| ω élevé | Actif intrinsèquement risqué |

---

## 10. Résumé

> **Volatilité actuelle = niveau structurel + choc récent + mémoire longue**

Le tout calibré via **maximum de vraisemblance gaussienne**.

---

### Extensions possibles

- Distribution Student-t  
- EGARCH / GJR-GARCH  
- Modèles hybrides ANN-GARCH  
